In [1]:
import urllib
from sqlalchemy import create_engine
import pandas as pd
import os
import pyodbc
import dotenv

# Load environment variables
dotenv.load_dotenv()

# Configure SQL Server connection string
params = urllib.parse.quote_plus(
    f"Driver={{{os.getenv('DB_DRIVER')}}};"
    f"Server={os.getenv('DB_SERVER')};"
    f"Database={os.getenv('DB_NAME')};"
    f"UID={os.getenv('DB_USER')};"
    f"PWD={os.getenv('DB_PASSWORD')};"
    f"Encrypt=no;"
    f"Connection Timeout=30;"
)

# إنشاء محرك SQLAlchemy مع تعطيل وضع المستقبل
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", future=True)




In [6]:
pyodbc.drivers()

['ODBC Driver 18 for SQL Server']

In [2]:
def Extract_df_From_Files(file):
    try:
        fileExtension = os.path.splitext(file)[1]
        if fileExtension == ".csv":
            df = pd.read_csv(file)
            return df
        elif fileExtension == ".parquet":
            df = pd.read_parquet(file)
            return df
        else :
            print(f" {file} The extension must be .csv or .parquet")
            return
    except Exception as e:
        print(f"Error is {e}")

def Create_table(df, name_table, engine):
    try:
        df.to_sql(name_table, engine, index=False, if_exists='replace')  # Use 'name' instead of 'file' to avoid ".csv" in table name
        print(f"Table {name_table} created successfully")
    except Exception as e:
        print(f"Can't create table because: {e}")


In [3]:
import os
path = "/home/elhossiny/Downloads/Python_ETL_Pipeline-20250204T145118Z-001/Python_ETL_Pipeline/DataSources"
files = os.listdir(path)
print(files)

['products.csv', 'stocks.csv', 'order_items.parquet', 'orders.csv', 'staffs.csv', 'stores.csv', 'brands.csv', 'customers.csv', 'categories.csv', 'order_items.csv']


In [4]:
for file in files:
    name = os.path.splitext(file)[0]
    df = Extract_df_From_Files(f"{path}/{file}")
    print("------------------------------------")

    if isinstance(df, pd.DataFrame):
        # Use the SQLAlchemy engine here
        print(df.shape,'\n', name, engine)
        Create_table(df, name, engine)
        print("Done")

------------------------------------
(334, 6) 
 products Engine(mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BServer%3Dlocalhost%2C1433%3BDatabase%3DE_commerce%3BUID%3Dsa%3BPWD%3DSTRONG123password%3BEncrypt%3Dno%3BConnection+Timeout%3D30%3B)
Table products created successfully
Done
------------------------------------
(939, 3) 
 stocks Engine(mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BServer%3Dlocalhost%2C1433%3BDatabase%3DE_commerce%3BUID%3Dsa%3BPWD%3DSTRONG123password%3BEncrypt%3Dno%3BConnection+Timeout%3D30%3B)
Table stocks created successfully
Done
------------------------------------
(4764, 6) 
 order_items Engine(mssql+pyodbc:///?odbc_connect=Driver%3D%7BODBC+Driver+18+for+SQL+Server%7D%3BServer%3Dlocalhost%2C1433%3BDatabase%3DE_commerce%3BUID%3Dsa%3BPWD%3DSTRONG123password%3BEncrypt%3Dno%3BConnection+Timeout%3D30%3B)
Table order_items created successfully
Done
------------------------------------
(1615, 8) 
 orders

In [5]:
query = """
SELECT TOP(10) * FROM order_items
"""
df_sql = pd.read_sql(query, engine)
print(df_sql)

   order_id item_id  product_id  quantity  list_price  discount
0         1       1          20         1      599.99      0.20
1         1       2           8         2     1799.99      0.07
2         1       3          10         2     1549.00      0.05
3         1       4          16         2      599.99      0.05
4         1       5           4         1     2899.99      0.20
5         2       1          20         1      599.99      0.07
6         2       2          16         2      599.99      0.05
7         3       1           3         1      999.99      0.05
8         3       2          20         1      599.99      0.05
9         4       1           2         2      749.99      0.10
